In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

##calculating stocks those trading currently above 200DMA and below 200DMA
import pandas as pd
import numpy as np
import csv
import glob
import os
import datetime
from pandas import ExcelWriter
import matplotlib

#File Path for datalab
#import gcsfs
#fs = gcsfs.GCSFileSystem(project='eco-composition-232001')

#with fs.open('case2-stockdatas/HPQ.csv') as f:
#     df = pd.read_csv(f)

#Capturing start time of the execution
startDT = datetime.datetime.now()
print ('Start Datetime:', str(startDT))

#path = r'datalab/full'
#path = fs.open('case2-stockdatas/')

#File path for local
path = r'C:\Users\Sudip\Sushant\Backup\fh_20190217\full_history'
all_files = glob.glob(os.path.join(path, "*.csv"))

#datalab files access
#all_files = fs.open('case2-stockdatas/*.csv')

gcol = ['date','symbol','close','200MA','close > 200MA']
lcol = ['date','symbol','close','200MA','close < 200MA']

dfG200DMA = pd.DataFrame(columns = gcol)
dfL200DMA = pd.DataFrame(columns = lcol)


##calculating stocks those trading currently above 200DMA and below 200DMA
for file_ in all_files:
    #printing filename to monitor the execution status
    print(file_)
    df = pd.read_csv(file_,header=0)
    df.date = pd.to_datetime((df.date))
    df['symbol'] = os.path.basename(file_)
    df.sort_values(by=["date"],inplace = True)
    df['50MA'] = df['close'].rolling(window=50).mean()
    df['200MA'] = df['close'].rolling(window=200).mean()
    df['close > 200MA'] = np.where(df['close'] > df['200MA'], 'Yes', 'No')
    df['close < 200MA'] = np.where(df['close'] < df['200MA'], 'Yes', 'No')
    df['50DMA > 200DMA'] = np.where(df['50MA'] > df['200MA'], 'Yes', 'No')
    df['50DMA < 200DMA'] = np.where(df['50MA'] < df['200MA'], 'Yes', 'No')
    df.sort_values(by=["date"],ascending=[False],inplace = True)
    #print(df) 
    for index, row in df.iterrows():
        
        if (index == 0) & (row['close > 200MA'] == 'Yes'):
            dfG200DMA = dfG200DMA.append(row[['date','symbol','close','200MA','close > 200MA']],ignore_index=True)
        elif (index == 0) & (row['close < 200MA'] == 'Yes'):
            dfL200DMA = dfL200DMA.append(row[['date','symbol','close','200MA','close < 200MA']],ignore_index=True)
    df.to_csv(file_, index=False)
    
#df.head(50).plot(x='symbol',y='200MA',kind='bar')
#df.head(50).plot(x='symbol',y='50MA',kind='bar')


print('There are',dfG200DMA.shape[0],'Stocks having closing price greater than 200DMA and they are listed below:')
dfG200DMA
print('\n')
print('There are',dfL200DMA.shape[0],'Stocks having closing price less than 200DMA and they are listed below:')
dfL200DMA

   
#Exporting data to excel to use for the research/operations
writer = ExcelWriter('GreaterThan200DMA.xlsx')
dfG200DMA.to_excel(writer,'Sheet1')
writer.save()

writer = ExcelWriter('LessThan200DMA.xlsx')
dfL200DMA.to_excel(writer,'Sheet1')
writer.save()

#calculating correlation between 50DMA and 200DMA
dfDMA = pd.DataFrame()
dfGldCross = pd.DataFrame()
cntr = 0
for file_ in all_files:
    df = pd.read_csv(file_,header=0)
    df['symbol'] = os.path.basename(file_)
    if df.shape[0] >= 200:
        df['50DMA'] = df['close'].rolling(window=50).mean()
        df['200DMA'] = df['close'].rolling(window=200).mean()
        df['50DMA > 200DMA'] = np.where(df['50DMA'] > df['200DMA'], 'Yes', 'No')
        dfDMA = dfDMA.append(df.loc[df['200DMA'] > 0,:])
        corr = dfDMA['50DMA'].corr(dfDMA['200DMA'])
        #print(cntr, os.path.basename(file_), corr)
    cntr = cntr + 1
    
#Calculating end time of execution
endDT = datetime.datetime.now()
print ('End Datetime:', str(endDT))

Start Datetime: 2019-03-19 13:07:17.243144
C:\Users\Sudip\Sushant\Backup\fh_20190217\full_history\A.csv
C:\Users\Sudip\Sushant\Backup\fh_20190217\full_history\AA.csv
C:\Users\Sudip\Sushant\Backup\fh_20190217\full_history\AAAU.csv
C:\Users\Sudip\Sushant\Backup\fh_20190217\full_history\AABA.csv
C:\Users\Sudip\Sushant\Backup\fh_20190217\full_history\AAC.csv
C:\Users\Sudip\Sushant\Backup\fh_20190217\full_history\AADR.csv
C:\Users\Sudip\Sushant\Backup\fh_20190217\full_history\AAL.csv
C:\Users\Sudip\Sushant\Backup\fh_20190217\full_history\AAMC.csv
C:\Users\Sudip\Sushant\Backup\fh_20190217\full_history\AAME.csv
C:\Users\Sudip\Sushant\Backup\fh_20190217\full_history\AAN.csv
C:\Users\Sudip\Sushant\Backup\fh_20190217\full_history\AAOI.csv
C:\Users\Sudip\Sushant\Backup\fh_20190217\full_history\AAON.csv
C:\Users\Sudip\Sushant\Backup\fh_20190217\full_history\AAP.csv
C:\Users\Sudip\Sushant\Backup\fh_20190217\full_history\AAPL.csv
C:\Users\Sudip\Sushant\Backup\fh_20190217\full_history\AAT.csv
C:\User

,date,symbol,close,200MA,close > 200MA
0,2019-03-01,A.csv,81.239998,67.705050,Yes
1,2019-03-01,AABA.csv,74.959999,68.193550,Yes
2,2019-03-01,AAME.csv,2.860000,2.683400,Yes
3,2019-03-01,AAN.csv,54.849998,47.276050,Yes
4,2019-03-01,AAON.csv,41.259998,36.501350,Yes
5,2019-03-01,AAT.csv,43.820000,39.060800,Yes
6,2019-03-01,AAXJ.csv,69.480003,69.075950,Yes
7,2019-03-01,AB.csv,29.549999,29.252650,Yes
8,2019-03-01,ABDC.csv,7.350000,6.487665,Yes
9,2019-03-01,ABG.csv,71.190002,69.697850,Yes




There are 3839 Stocks having closing price less than 200DMA and they are listed below:


,date,symbol,close,200MA,close < 200MA
0,2019-03-01,AA.csv,29.660000,38.087600,Yes
1,2019-03-01,AAC.csv,2.870000,6.358150,Yes
2,2019-03-01,AADR.csv,45.930000,49.410150,Yes
3,2019-03-01,AAL.csv,34.639999,37.385700,Yes
4,2019-03-01,AAMC.csv,39.000000,52.446900,Yes
5,2019-03-01,AAOI.csv,13.240000,28.683125,Yes
6,2019-03-01,AAPL.csv,174.970001,191.128850,Yes
7,2019-03-01,AAU.csv,0.630000,0.656450,Yes
8,2019-03-01,AAWW.csv,54.000000,58.725750,Yes
9,2019-03-01,AAXN.csv,54.450001,58.591750,Yes


C:\Users\Sudip\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
